# DOPASOWANI WZORCA DO TEKSTU

In [2]:
filename = "pan-tadeusz.txt"
ALPHABET = set()
with open(filename, "r") as file:
    T = file.read()
    ALPHABET.update(T)
print(ALPHABET)
ALGORITHMS = dict()
Names = ["naiwny", "automatów skończonych", "KMP"]

{'s', 'ż', 'Ł', '5', '(', '6', 'ę', '—', 'v', 'u', 'D', 'ą', 'M', 'K', 'P', '/', 'ń', 'x', '\n', '.', '-', '*', '8', '7', 'N', 'w', 'y', 'b', 'ź', 'a', 'U', 'T', 'I', 'ć', 'ł', 'd', 'S', 'à', '4', 'Ć', 'l', 'R', 'V', '…', 'ó', 'F', 'h', 'Ó', 'e', 'O', 'p', 'ś', '2', ' ', ',', '!', 'Z', 'W', 'r', 'é', 'A', 'Ż', 'j', 'C', '3', 'k', 'z', '»', 'Ź', 'q', 'f', 'Ś', '1', 't', 'H', ';', 'J', ':', 'E', 'i', 'g', '?', 'm', 'n', 'G', ')', 'L', '9', '«', 'B', 'o', 'c', 'æ', '0'}


##      Algorytm naiwny

In [3]:
def findPattern(T, P):
    n = len(T)
    m = len(P)
    S = []
    for i in range(n-m+1):
        if T[i:i+m] == P:
            S.append(i)
    return S

ALGORITHMS[Names[0]] = findPattern
print(findPattern("ala ala ala", "ala"))

[0, 4, 8]


## Algorytm automat skończonych

In [4]:
def findMatch(P, a, i):
    m = len(P)
    Pi = P[:i] + a
    m2 = len(Pi)
    for i in range(min(m, m2), 0, -1):
        if Pi[m2-i:] == P[:i]:
            return i
    return 0            

def makeAutomaton(P):
    m = len(P)
    my_alphabet = set(P)
    return [{a:findMatch(P, a, i) for a in my_alphabet} for i in range(m + 1)]

def _findPatternAutomat(T, m, A):
    state = 0
    S=[]
    for i, letter in enumerate(T):
        state = A[state].get(letter, 0)
        if state == m:
            S.append(i-m + 1)
    return S


def findPattern(T, P):
    A = makeAutomaton(P)
    return _findPatternAutomat(T, len(P), A)

ALGORITHMS[Names[1]] = findPattern
print(findPattern("ala ala ala", "ala"))

[0, 4, 8]


## Algorytm KMP

In [4]:
def prefixFunction(P):
    res = [0]
    k = 0
    for j in range(1, len(P)):
        while k > 0 and P[k] != P[j]:
            k = res[k-1]
        if P[k] == P[j]:
            k += 1
        res.append(k)
    return res


def _findPatternKMP(T, P, pi):
    S = []
    m = len(P)
    q = 0
    for i in range(len(T)):
        while q > 0 and T[i] != P[q]:
            q = pi[q-1]
        if T[i] == P[q]:
            q += 1 
        if q == m:
            q = pi[q-1]
            S.append(i - m + 1)
    return S

def findPattern(T, P):
    pi = prefixFunction(P)
    return _findPatternKMP(T, P, pi)

ALGORITHMS[Names[2]] = findPattern
print(findPattern("ala ala ala", "ala"))

[0, 4, 8]


## Testy wydajnościowe
### w plikach test1.txt, test2.txt, test3.txt w pierwszej linii są zapisane wzorce a w kolejnych tekst w którym ich szukamy

In [44]:
import time as timelib
LINE = "---------------------------------------------------------------------------------------------------------"
def _time(function, args):
    tmp = function.__defaults__
    function.__defaults__ = args
    start = timelib.time()
    res = function()
    end = timelib.time()
    function.__defaults__ = tmp
    return round(100 *(end - start), 4), res

def _print_res(res):
    n = len(res)
    if n > 10:
        print(f"\twynik:\t{res[:10]}......")
    else:
        print(f"\twynik:\t{res}")

def RunTest(test_file_name, preprocesing = None, algoritm = None):
    with open(test_file_name, "r") as test:
        Patterns = test.readline().split()
        Text = test.read()
    print(f"n={len(Text)}")
    print(LINE)
    for P in Patterns:
        if(len(P)>10):
            print(f"Pattern={P[:10]}...")
        else:
            print(f"Pattern={P}")
        print(f"m={len(P)}")
        print("\tALGORYTM NAIWNY")
        print("\tpreprocesing:\t0ms")
        t, res = _time(ALGORITHMS["naiwny"], (Text, P))
        print(f"\tszukanie wzorców:\t{t}ms")
        print(f"\tsumarycznie:\t{t}ms")
        _print_res(res)

        print()
        print("\tALGORYTM AUTOMATÓW SKOŃCZONYCH")
        t1, A = _time(makeAutomaton, (P,))
        print(f"\tpreprocesing:\t{t1}ms")
        t2, res = _time(_findPatternAutomat, (Text, len(P), A))
        print(f"\tszukanie wzorców:\t{t2}ms")
        print(f"\tsumarycznie:\t{t1 + t2}ms")
        _print_res(res)

        print()
        print("\tALGORYTM KMP")
        t3, pi = _time(prefixFunction, (P,))
        print(f"\tpreprocesing:\t{t3}ms")
        t4, res = _time(_findPatternKMP, (Text, P, pi))
        print(f"\tszukanie wzorców:\t{t4}ms")
        print(f"\tsumarycznie:\t{t3 + t4}ms")
        _print_res(res)

        print(LINE)
        if preprocesing is not None:
            preprocesing.append((0, t1, t3))
        if algoritm is not None:
            algoritm.append((t, t2, t4))

In [58]:
testy = ["test1.txt", "test2.txt", "test3.txt"]
for test in testy:
    print(test)
    RunTest(test)
    print("\n\n\n\n")

test1.txt
n=199
---------------------------------------------------------------------------------------------------------
Pattern=aa
m=2
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	0.0ms
	sumarycznie:	0.0ms
	wynik:	[50, 53, 64, 65, 77, 78, 167, 168, 186]

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	0.0ms
	sumarycznie:	0.0ms
	wynik:	[50, 53, 64, 65, 77, 78, 167, 168, 186]

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	0.0ms
	sumarycznie:	0.0ms
	wynik:	[50, 53, 64, 65, 77, 78, 167, 168, 186]
---------------------------------------------------------------------------------------------------------
Pattern=abbab
m=5
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	0.0ms
	sumarycznie:	0.0ms
	wynik:	[79, 123]

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	0.0ms
	sumarycznie:	0.0ms
	wynik:	[79, 123]

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	0.0995ms
	sumarycznie:	0.0995ms
	wynik:	[79, 123]
-------------

## Badanie algorytmów na podstawie Pana Tadeusza

In [45]:
preprocesingTime = []
matchingTime = []
RunTest("pan-tadeusz.txt", preprocesing=preprocesingTime, algoritm=matchingTime)

n=447342
---------------------------------------------------------------------------------------------------------
Pattern=pan
m=3
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	22.6082ms
	sumarycznie:	22.6082ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	13.8516ms
	sumarycznie:	13.8516ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	12.2024ms
	sumarycznie:	12.2024ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......
---------------------------------------------------------------------------------------------------------
Pattern=pani
m=4
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	14.6317ms
	sumarycznie:	14.6317ms
	wynik:	[129, 1254, 7995, 14647, 18326, 18797, 27375, 33394, 34461, 34487]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorc

In [46]:
RunTest("pan-tadeusz.txt", preprocesing=preprocesingTime, algoritm=matchingTime)

n=447342
---------------------------------------------------------------------------------------------------------
Pattern=pan
m=3
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	21.3424ms
	sumarycznie:	21.3424ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	16.8276ms
	sumarycznie:	16.8276ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	14.4044ms
	sumarycznie:	14.4044ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......
---------------------------------------------------------------------------------------------------------
Pattern=pani
m=4
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	13.6914ms
	sumarycznie:	13.6914ms
	wynik:	[129, 1254, 7995, 14647, 18326, 18797, 27375, 33394, 34461, 34487]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorc

In [27]:
RunTest("pan-tadeusz.txt", preprocesing=preprocesingTime, algoritm=matchingTime)

n=447342
---------------------------------------------------------------------------------------------------------
Pattern=pan
m=3
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	47.7224ms
	sumarycznie:	47.7224ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	39.4499ms
	sumarycznie:	39.4499ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	47.4937ms
	sumarycznie:	47.4937ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......
---------------------------------------------------------------------------------------------------------
Pattern=pani
m=4
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	25.2173ms
	sumarycznie:	25.2173ms
	wynik:	[129, 1254, 7995, 14647, 18326, 18797, 27375, 33394, 34461, 34487]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorc

In [28]:
RunTest("pan-tadeusz.txt", preprocesing=preprocesingTime, algoritm=matchingTime)

n=447342
---------------------------------------------------------------------------------------------------------
Pattern=pan
m=3
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	50.8393ms
	sumarycznie:	50.8393ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0511ms
	szukanie wzorców:	24.7452ms
	sumarycznie:	24.796300000000002ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	30.1308ms
	sumarycznie:	30.1308ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......
---------------------------------------------------------------------------------------------------------
Pattern=pani
m=4
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	33.0441ms
	sumarycznie:	33.0441ms
	wynik:	[129, 1254, 7995, 14647, 18326, 18797, 27375, 33394, 34461, 34487]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	

In [29]:
RunTest("pan-tadeusz.txt", preprocesing=preprocesingTime, algoritm=matchingTime)

n=447342
---------------------------------------------------------------------------------------------------------
Pattern=pan
m=3
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	38.09ms
	sumarycznie:	38.09ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	27.2814ms
	sumarycznie:	27.2814ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......

	ALGORYTM KMP
	preprocesing:	0.0ms
	szukanie wzorców:	27.1514ms
	sumarycznie:	27.1514ms
	wynik:	[129, 1254, 2227, 7013, 7077, 7713, 7995, 8167, 8189, 9210]......
---------------------------------------------------------------------------------------------------------
Pattern=pani
m=4
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	36.1662ms
	sumarycznie:	36.1662ms
	wynik:	[129, 1254, 7995, 14647, 18326, 18797, 27375, 33394, 34461, 34487]......

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	0.0ms
	szukanie wzorców:	

## analiza wyników:

In [30]:
print(preprocesingTime,"\n", matchingTime)

[(0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0511, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0), (0, 0.0, 0.0)] 
 [(132.3735, 82.322, 89.8772), (79.2989, 55.7895, 68.8774), (66.876, 57.9093, 43.3915), (36.5623, 25.5426, 20.7438), (47.7224, 39.4499, 47.4937), (25.2173, 22.1644, 21.58), (50.8393, 24.7452, 30.1308), (33.0441, 28.4235, 27.4617), (38.09, 27.2814, 27.1514), (36.1662, 25.0726, 30.7678)]


In [38]:
n = len(matchingTime)
preprocesing = {Names[i]: [t[i] for t in preprocesingTime] for i in range(3)}
matching = {Names[i]: [t[i] for t in matchingTime] for i in range(3)}


print(f"{Names[0]}\t\t\t\t{Names[1]}\t\t{Names[2]}")
print("pre-proc\tmatching\t"*3)
for i in range(n):
    for j in range(3):
        print(preprocesing[Names[j]][i], matching[Names[j]][i], sep="\t ", end="\t ")
    print()

naiwny				automatów skończonych		KMP
pre-proc	matching	pre-proc	matching	pre-proc	matching	
0	 132.3735	 0.0	 82.322	 0.0	 89.8772	 
0	 79.2989	 0.0	 55.7895	 0.0	 68.8774	 
0	 66.876	 0.0	 57.9093	 0.0	 43.3915	 
0	 36.5623	 0.0	 25.5426	 0.0	 20.7438	 
0	 47.7224	 0.0	 39.4499	 0.0	 47.4937	 
0	 25.2173	 0.0	 22.1644	 0.0	 21.58	 
0	 50.8393	 0.0511	 24.7452	 0.0	 30.1308	 
0	 33.0441	 0.0	 28.4235	 0.0	 27.4617	 
0	 38.09	 0.0	 27.2814	 0.0	 27.1514	 
0	 36.1662	 0.0	 25.0726	 0.0	 30.7678	 


In [41]:
for i in range(3):
    time = [preprocesing[Names[i]][j] + matching[Names[i]][j] for j in range(n)]
    print(f"{Names[i]}:")
    print(f"min:{min(time)}")
    print(f"max:{max(time)}")
    print(f"avg:{(sum(time)/len(time))}")
    print()

naiwny:
min:25.2173
max:132.3735
avg:54.61900000000001

automatów skończonych:
min:22.1644
max:82.322
avg:38.875150000000005

KMP:
min:20.7438
max:89.8772
avg:40.747530000000005



Najgorzej radzi sobie algorytm naiwny, pozostałe dwa mają podobny czas działania, czasy preprocesingu jedynie w jednym przypadku były różne od zera, wyjątkowym przypadkiem było tworzenie w 4 wywołaniu automatu dla wzorca "pan" trwał 0.0511ms

## Proponowany test dla którego algorytm naiwy jest 5 razy dłuższy

In [83]:
with open("test4.txt", "w") as test:
    P = "abc"*1000+"c"
    test.write(P + "\n")
    test.write("abcdefghij"*100000 + P + "asdgdgsba"*100000 + P + "ba" + P)

RunTest("test4.txt")

n=1909005
---------------------------------------------------------------------------------------------------------
Pattern=abcabcabca...
m=3001
	ALGORYTM NAIWNY
	preprocesing:	0ms
	szukanie wzorców:	387.43ms
	sumarycznie:	387.43ms
	wynik:	[1000000, 1903001, 1906004]

	ALGORYTM AUTOMATÓW SKOŃCZONYCH
	preprocesing:	1357.924ms
	szukanie wzorców:	77.8903ms
	sumarycznie:	1435.8143ms
	wynik:	[1000000, 1903001, 1906004]

	ALGORYTM KMP
	preprocesing:	0.2224ms
	szukanie wzorców:	98.8036ms
	sumarycznie:	99.026ms
	wynik:	[1000000, 1903001, 1906004]
---------------------------------------------------------------------------------------------------------


aby stworzyć taki test trzeba wydłurzać długość wzorca oraz umieścić wzorzc, lub znaczne jego części w tekscie, wydłurzy to operacje przyrównania napisów, trzeba to zrobić dla napisów długośći rzędu $10^3$ ponieważ w pythonie operacja s1 == s2 jest realizowana szybko i dla mniejszych długości jest porónywalna z operacją s1[i] == s2[j]

## Wnioski:
najmniej urzytecznym algorytmem jest ten który wykorzystuje automat skończony, gdyż czas przygotowania automaty jest rzędu O(m^4).

Algorytm naiwny radzi sobie zadziwiająco dobrz, wydaje mi się że jest to spowodowane urzysiem operacji s1 == s2 która jest zaimplementowana w C

Algorytm KMP byłby najleprzym wyborem gdyby operacje s1[i] == s2[j] były proporcjonalne do  czasu wykonania s1 == s2 podzielonego przez min(|s1|, |s2|)